In [1]:
# 📦 Imports nécessaires
import pandas as pd

In [12]:
# 📂 Chargement des données
df = pd.read_csv("output/jobs_infos.csv")

In [13]:
# 🧼 Nettoyage basique des TJM
df["TJM_min"] = df["TJM"].str.extract(r'(\d+)\s*€')
df["TJM_max"] = df["TJM"].str.extract(r'(\d+)[^0-9]+(\d+)').iloc[:,1]
df["TJM_min"] = pd.to_numeric(df["TJM_min"], errors="coerce")
df["TJM_max"] = pd.to_numeric(df["TJM_max"], errors="coerce")
df["TJM_moyen"] = df[["TJM_min", "TJM_max"]].mean(axis=1)

# 🧼 Nettoyage des durées
df["DURATION_clean"] = df["DURATION"].str.extract(r'(\d+)').astype(float)

# 📊 Statistiques générales
print("📌 Nombre total d'offres :", len(df))
print("\n📌 TJM moyen (hors NaN) :", round(df["TJM_moyen"].mean(), 2), "€")

# 📌 Durée moyenne (en mois)
print("\n📌 Durée moyenne (en mois) :", round(df["DURATION_clean"].mean(), 2))

# 📌 Répartition par expérience
print("\n📌 Répartition des niveaux d'expérience :")
print(df["EXPERIENCE"].value_counts())

# 📌 Répartition géographique
print("\n📌 Répartition géographique :")
print(df["LOCATION"].value_counts())

# 📌 Aperçu des offres triées par TJM
print("\n📌 Top offres par TJM moyen :")
print(df[["TITLE", "TJM_moyen"]].sort_values(by="TJM_moyen", ascending=False))

# 📌 Missions sans TJM ou durée renseignée
print("\n📌 Offres sans TJM ou durée renseignée :")
print(df[df["TJM_moyen"].isna() | df["DURATION_clean"].isna()][["TITLE", "TJM", "DURATION"]])

📌 Nombre total d'offres : 64

📌 TJM moyen (hors NaN) : 468.71 €

📌 Durée moyenne (en mois) : 9.92

📌 Répartition des niveaux d'expérience :
EXPERIENCE
5 à 10 ans d’expérience    43
2 à 5 ans d’expérience     13
> 10 ans d’expérience       7
< 2 ans d’expérience        1
Name: count, dtype: int64

📌 Répartition géographique :
LOCATION
Paris, France                                  17
Paris, Île-de-France                            6
Saint-Herblain, Pays de la Loire                5
France                                          4
Lille, Hauts-de-France                          3
Toulouse, Occitanie                             3
Lyon, Auvergne-Rhône-Alpes                      3
Montpellier, Occitanie                          2
Provence-Alpes-Côte d'Azur, France              1
Bagneux, Île-de-France                          1
Gémenos, Provence-Alpes-Côte d'Azur             1
Île-de-France, France                           1
Grand Est, France                               1
Aix-en-Provenc

In [19]:
# 🔍 Définir le skill à chercher
skill = "Javascript"

# 🧼 Nettoyer un peu le texte : on peut tout passer en minuscule pour éviter les problèmes de casse
df["SKILLS_str"] = df["SKILLS"].fillna("").str.lower()

# 🔍 Filtrer les lignes contenant le mot-clé (skill)
filtered = df[df["SKILLS_str"].str.contains(skill.lower())]

# 🖨️ Résultat
print(f"\n🎯 Offres contenant le skill '{skill}':")
print(filtered[["TITLE", "SKILLS", "LOCATION"]])


🎯 Offres contenant le skill 'Javascript':


,Unnamed: 0,JOB_URL,MISSION,TITLE,SKILLS,DATE,DURATION,TJM,EXPERIENCE,REMOTE,LOCATION,TJM_min,TJM_max,TJM_moyen,DURATION_clean,SKILLS_str
0,0,https://www.free-work.com/fr/tech-it/directeur...,Mission freelance,UX/UI Designer,"['Design system', 'Figma']",Dès que possible,8 mois,450-550 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Paris, France",550.0,550.0,550.0,8.0,"['design system', 'figma']"
1,1,https://www.free-work.com/fr/tech-it/consultan...,Mission freelance,Consultant SAP ABAP S/4 PI/PO SCPI 100% Remote,['SAP'],Dès que possible,6 mois,350-400 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Montpellier, Occitanie",400.0,400.0,400.0,6.0,['sap']
2,2,https://www.free-work.com/fr/tech-it/administr...,Mission freelance,"ELK Senior Consultant: Logboard, Stack Managem...",[],20/04/2025,12 mois,650-750 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Paris, Île-de-France",750.0,750.0,750.0,12.0,[]
3,3,https://www.free-work.com/fr/tech-it/consultan...,Mission freelance,Lead Architect: Initiative and Team Leadership...,[],05/05/2025,9 mois,750-850 €⁄j,> 10 ans d’expérience,Télétravail 100%,"Paris, Île-de-France",850.0,850.0,850.0,9.0,[]
4,4,https://www.free-work.com/fr/tech-it/developpe...,Mission freelance,Développeur Rust - Full remote - Anglais fluent,"['Python', 'Rust']",Dès que possible,8 mois,400-550 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Paris, France",550.0,550.0,550.0,8.0,"['python', 'rust']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,https://www.free-work.com/fr/tech-it/developpe...,Mission freelance,Offshoring - Angular / Java - 1h max de décala...,"['Angular', 'Java']",Dès que possible,3 ans,100-230 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Paris, France",230.0,230.0,230.0,3.0,"['angular', 'java']"
60,60,https://www.free-work.com/fr/tech-it/administr...,Mission freelance,DEVIENS #AUTEUR Services de virtualisation de ...,"['Hyper-v', 'Virtualisation', 'Windows Server']",Dès que possible,8 mois,100-300 €⁄j,2 à 5 ans d’expérience,Télétravail 100%,"Saint-Herblain, Pays de la Loire",300.0,300.0,300.0,8.0,"['hyper-v', 'virtualisation', 'windows server']"
61,61,https://www.free-work.com/fr/tech-it/consultan...,Mission freelance,Consultant SAP ISH,['SAP'],Dès que possible,5 jours,NaN,5 à 10 ans d’expérience,Télétravail 100%,"Lyon, Auvergne-Rhône-Alpes",NaN,NaN,NaN,5.0,['sap']
62,62,https://www.free-work.com/fr/tech-it/business-...,Mission freelance,Business analyst SAP IBP / PP,['SAP'],Dès que possible,1 an,590-710 €⁄j,5 à 10 ans d’expérience,Télétravail 100%,"Nantes, Pays de la Loire",710.0,710.0,710.0,1.0,['sap']
